# Notebook for scraping CCS data information from TADA site as well as Creating Interactive Sampling Map

`
By Oscar Lares for the Smart Mobility and Infrastructure Lab - 3/19/2024
`

## Import Libraries for Scraping

In [1]:
#import libraries for scraping and reading CSV files

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import os
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Obtaining CCS names from CSV files

### params

In [2]:
# change these to your PATHS!
ccs_portables_path = r"F:\Data\CCS\Portable CCS Station Data" #path to portables CCS station data main folder
ccs_permanent_path = r"F:\Data\CCS\Permanent CCS Station Data\Lane Exports" #path to permanent CCS station data main folder

#define column name so that you dont need to read in all columns from the CSV
column_names = ['Site','Volume']

### Function to get CCS names from CSV files

In [3]:
def get_unique_CCS_sites(path=r"F:\Data\CCS\Portable CCS Station Data", column_names=column_names):

    #init list to store unique site names
    unique_site_values = set()

    #loop through each folder year in the main path given
    for year in os.listdir(path):
        print('--------------------------')
        print(f'Getting data from {year}')
        print('--------------------------')
        year_path = os.path.join(path, year)

        #check if its a directory
        if os.path.isdir(year_path):

            #Now use glob to iterate through all the folders in the sub directory containing data for that year
            csv_files = glob.glob(os.path.join(year_path, '*.csv'))

            #now iterate through each CSV file since we have a list of filenames
            for csv_file in csv_files:
                print(f'Getting site values from file {csv_file}. . .')

                #read into pandas
                df = pd.read_csv(csv_file, usecols = column_names)

                #eliminate any rows where 'Volume' is NaN
                df = df.dropna(subset=['Volume'])

                #get unique values from df
                current_sites_from_df = set(df['Site'].unique())

                #add only new sites to the set of all unique sites
                unique_site_values.update(current_sites_from_df)

    #once loop is done running, convert the set to a list to obtain final list of all CCS sites present in CSV files
    unique_site_values_list = list(unique_site_values)
    print('##############################')
    print('All done!')

    return unique_site_values_list


### CCS Portables

In [4]:
CCS_portables_list = get_unique_CCS_sites(path=ccs_portables_path, column_names=column_names)

--------------------------
Getting data from 2021
--------------------------
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-12.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-01.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-02.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-03.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-04.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-05.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-06.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-07.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-08.csv. . .
Getting site values from file F:\Data\CCS\Portable CCS Station Data\2021\2021-09.csv. . .
Getting site values fro

### CCS Permanents

In [ ]:
CCS_permanent_list = get_unique_CCS_sites(path=ccs_permanent_path, column_names=column_names)

## Scraping CCS from TADA site (if needed)

### URL generation function

In [ ]:
def generate_url(site_id, site_type):
    base_url = "https://gdottrafficdata.drakewell.com/sitedashboard.asp?"
    
    # Remove the dash
    clean_site_id = site_id.replace('-', '')
    
    if site_type == "PORTABLES":
        # Insert the underscore before the last 4 digits, then pad with leading zeroes
        cosit_formatted = clean_site_id[:-4] + "_" + clean_site_id[-4:]
        cosit_formatted = cosit_formatted.zfill(12)  # 12 digits (including underscore)
        node_value = "GDOT_PORTABLES"

    elif site_type == "CCS":
        # Pad with leading zeroes to make the total length 12
        cosit_formatted = clean_site_id.zfill(12)
        node_value = "GDOT_CCS"
    else:
        raise ValueError("Invalid site type. Must be 'PORTABLES' or 'CCS'.")

    url = f"{base_url}node={node_value}&cosit={cosit_formatted}"
    return url

#### Testing

In [ ]:
# Test usage:
portables_url = generate_url("139-0183", "CCS")
ccs_url = generate_url("097-r807", "PORTABLES")

print(portables_url)
print(ccs_url)

### Site scraping function

In [ ]:
def scrape_sites(sites, site_type, webdriver_path, save_interval=100, output_file='scraped_data.csv'):

    errors = []
    service = Service(executable_path=webdriver_path)
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Chrome/88.0.4324.150")
    driver = webdriver.Chrome(service=service, options=options)
    data_list = [] #list to store site data dicts

    for i, site_id in enumerate(sites):
        try:
            # Generate the URL based on the site_id and site_type
            url = generate_url(site_id, site_type)  # generate url from function
            print(url)
            driver.get(url)

            # Scraping
            # lrs_element = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'LRS section:')]/following-sibling::span"))
            # ).text

            # city_element = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'City:')]/following-sibling::span"))
            # ).text    

            county_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'County:')]/following-sibling::span"))
            ).text

            functional_class_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Functional class:')]/following-sibling::span"))
            ).text

            coordinates_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Coordinates:')]/following-sibling::span"))
                ).text

            # add scraped data to dict
            data_row = {                
                'Site ID': site_id,
                'County': county_element,
                'Functional Class': functional_class_element,
                'Coordinates': coordinates_element
                }
             #append current dict of data to the data list
            data_list.append(data_row)

        except Exception as e:
            print(f"Error scraping site {site_id}: {e}")
            errors.append({'Site ID': site_id, 'Error': str(e)})

        finally:
            # Periodic save of both data and errors
            if i % save_interval == 0 or i == len(sites) - 1:
                checkpoint_df = pd.DataFrame(data_list)
                checkpoint_df.to_csv(f'{output_file}_partial.csv', index=False)
                # Save the error list as well
                pd.DataFrame(errors).to_csv(f'error_sites_{site_type}.csv', index=False)
                print(f"Progress and errors saved at site {i}.")

    driver.quit()

    # Convert final list to DataFrame and save
    final_df = pd.DataFrame(data_list)
    final_df.to_csv(output_file, index=False)
    print("Final data saved.")

    # Save the final error list
    pd.DataFrame(errors).to_csv(f'error_sites_{site_type}_final.csv', index=False)
    print(f"Final list of errors saved. Check 'error_sites_{site_type}_final.csv'.")

    return final_df, errors

### Parameters

In [ ]:
chromedriver_path = r"C:\Program Files\Webdrivers\chromedriver-win64\chromedriver.exe"
save_interval = 100
permanent_sites_output_filename = 'CCS_permanent_sites_information.csv'
portable_sites_output_filename = 'CCS_portable_sites_information.csv'

### Permanent CCS site scraping

In [ ]:
final_df, error_sites = scrape_sites(CCS_permanent_list, 'CCS', chromedriver_path, save_interval=save_interval, output_file=permanent_sites_output_filename)

### Portable CCS site scraping

In [ ]:
final_portables_df, portables_error_sites = scrape_sites(CCS_portables_list, 'PORTABLES', chromedriver_path, save_interval=save_interval, output_file=portable_sites_output_filename)

## Filter Data and Create Interactive Map for Sampling

### import libraries

In [6]:
import geopandas as gpd

from shapely.geometry import Point

### functions to use

In [7]:
def parse_duration(duration_str):
    parts = duration_str.strip().split()

    #Assuming format is in the form "X hours Y minutes"
    total_minutes = 0

    if 'hour' in duration_str:
        total_minutes += int(parts[0]) * 60 #converting the hour to minutes
        total_minutes += int(parts[2]) if 'minute' in duration_str else 0
    
    else:
        total_minutes += int(parts[0])
    return total_minutes

# Function to read and preprocess and match RITIS events to CCS sites
def match_ritis_events(events_gdf, ccs_gdf, radius=1600, predicate='within'):
    geoevents = events_gdf[['geometry']]
    geosites = ccs_gdf[['geometry', 'TC_NUM']]

    # Convert the CRS to a projected CRS for distance calculations
    geoevents = geoevents.to_crs('EPSG:3857')
    geosites = geosites.to_crs('EPSG:3857')

    print('CRS for events:', events_gdf.crs)
    print('CRS for sites:', ccs_gdf.crs)
    
    buffered_sites = geosites.geometry.buffer(radius)
    buffered_sites = gpd.GeoDataFrame(geometry=buffered_sites, crs='EPSG:3857') #convert back to geodf 

    # Perform spatial join to find crashes within the buffered sites
    events_in_radius = gpd.sjoin(geoevents, buffered_sites, predicate=predicate)

    # Use this index to map back to the 'TC_NUM' or the site identifier
    events_in_radius['CCS_Pair'] = events_in_radius['index_right'].map(geosites['TC_NUM'])

    # Drop the 'index_right' as it's no longer needed
    events_in_radius = events_in_radius.drop(columns=['index_right'])

    # Convert back to the original CRS
    events_in_radius = events_in_radius.to_crs('EPSG:4326')

    # Merge 'CCS_Pair' information back into the original events_gdf using the index
    events_gdf = events_gdf.merge(events_in_radius[['CCS_Pair']], left_index=True, right_index=True, how='left')
    events_gdf = events_gdf.dropna(subset=['CCS_Pair'])
    print('Size of events_gdf with CCS_Pair: ', events_gdf.shape)

    return events_gdf

def filter_events_to_counties(ritis_file_path, shp_path):

    events_within_counties_chunks = []
    counties = gpd.read_file(shp_path)

    #load events data
    for year in os.listdir(ritis_file_path):
        print('--------------------------')
        print(f'Getting data from {year}')
        print('--------------------------')
        year_path = os.path.join(ritis_file_path, year)

        csv_files = glob.glob(os.path.join(year_path, '*.csv'))

        for file in csv_files:
            print(f'Getting event data from file {file}. . .')
            events_df = pd.read_csv(file)
            events_df = events_df[(events_df['Agency-specific Type'] == 'Accident') | (events_df['Agency-specific Type'] == 'Fire') | (events_df['Agency-specific Type'] == 'Debris')]
            events_df['Duration_minutes'] = events_df['Duration (Incident clearance time)'].apply(parse_duration)
            events_df = events_df[events_df['Duration_minutes'] > 20]

            events_gdf = gpd.GeoDataFrame(events_df, geometry=gpd.points_from_xy(events_df.Longitude, events_df.Latitude))
            events_gdf.crs = counties.crs

            events_filtered = gpd.sjoin(events_gdf, counties, how='inner', predicate='within')
            events_filtered.drop(['index_right'], axis=1, inplace=True)

            events_within_counties_chunks.append(events_filtered)

            print(f'Event data for file {file[-11:]} appended')

    events_within_counties = pd.concat(events_within_counties_chunks)

    return events_within_counties

def filter_sites_to_counties(filtered_sites, shp_path):

    counties = gpd.read_file(shp_path)
    filtered_sites_gdf = gpd.GeoDataFrame(filtered_sites, geometry=gpd.points_from_xy(filtered_sites.LONG, filtered_sites.LAT))

    filtered_sites_gdf.crs = counties.crs

    sites_within_counties = gpd.sjoin(filtered_sites_gdf, counties, how='inner', predicate='within')
    sites_within_counties.drop(['index_right'], axis=1, inplace=True)

    return sites_within_counties

def calculate_distances(joined_gdf, ccs_gdf):
    # Create an empty DataFrame to store the closest CCS site and distance for each event
    closest_sites_info = pd.DataFrame(columns=['EventID', 'ClosestCCS', 'Distance'])

    # Group by the event's original index to handle potential multiple matches
    grouped = joined_gdf.groupby(joined_gdf.index)

    for event_id, matches in grouped:
        # Calculate the distance from the event to each CCS site in the matches
        distances = matches.apply(lambda match: ccs_gdf.loc[match['index_right']].geometry.distance(match['geometry']), axis=1)
        # Get the index of the closest CCS site
        closest_site_idx = distances.idxmin()
        # Get the ID of the closest CCS site
        closest_ccs_id = matches.loc[closest_site_idx]['index_right']
        # Get the minimum distance
        min_distance = distances.min()
        # Append to the DataFrame
        closest_sites_info = closest_sites_info.append({
            'EventID': event_id,
            'ClosestCCS': closest_ccs_id,
            'Distance': min_distance
        }, ignore_index=True)

    return closest_sites_info

### Filter CCS sites and RITIS events to only cover in south GA

#### define RITIS event path and Shape file path for counties

In [8]:
ritis_events_path = r'F:\Data\RITIS Event Data'
shp_path = r'D:\OneDrive - University of Georgia\Research\Fall 2023\RP 23-25 - Probe Project\SHP files\Counties_Georgia.shp'
site_locations_path = r"F:\Data\CCS\Site Locations 2023.xlsx"

#### read in site location data

In [9]:
site_locations = pd.read_excel(site_locations_path, sheet_name='Short-term')
# site_locations

#### filter down site locations to only include sites we have data for

In [10]:
filtered_sites = site_locations[site_locations['TC_NUM'].isin(CCS_portables_list)]

#### Get all events falling within South GA counties we want to look at 

In [11]:
events_within_counties = filter_events_to_counties(ritis_events_path, shp_path)

--------------------------
Getting data from 2022
--------------------------
Getting event data from file F:\Data\RITIS Event Data\2022\01-2022.csv. . .
Event data for file 01-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\02-2022.csv. . .
Event data for file 02-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\03-2022.csv. . .
Event data for file 03-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\04-2022.csv. . .
Event data for file 04-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\05-2022.csv. . .
Event data for file 05-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\06-2022.csv. . .
Event data for file 06-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\07-2022.csv. . .
Event data for file 07-2022.csv appended
Getting event data from file F:\Data\RITIS Event Data\2022\08-2022.csv. . .
Event data for file 08-2022.

C:\Users\Oscar\AppData\Local\Temp\ipykernel_13584\3069651871.py:65: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  events_df = pd.read_csv(file)


Event data for file 08-2023.csv appended
Getting event data from file F:\Data\RITIS Event Data\2023\09-2023.csv. . .
Event data for file 09-2023.csv appended
Getting event data from file F:\Data\RITIS Event Data\2023\10-2023.csv. . .
Event data for file 10-2023.csv appended
Getting event data from file F:\Data\RITIS Event Data\2023\11-2023.csv. . .
Event data for file 11-2023.csv appended
Getting event data from file F:\Data\RITIS Event Data\2023\12-2023.csv. . .
Event data for file 12-2023.csv appended
--------------------------
Getting data from 2021
--------------------------
Getting event data from file F:\Data\RITIS Event Data\2021\04-2021.csv. . .
Event data for file 04-2021.csv appended
Getting event data from file F:\Data\RITIS Event Data\2021\05-2021.csv. . .
Event data for file 05-2021.csv appended
Getting event data from file F:\Data\RITIS Event Data\2021\06-2021.csv. . .
Event data for file 06-2021.csv appended
Getting event data from file F:\Data\RITIS Event Data\2021\07-2

#### Get all sites falling within South GA counties we want to look at

In [12]:
sites_within_counties = filter_sites_to_counties(filtered_sites, shp_path)

#### Match RITIS events to CCS sites

In [13]:
matched_events = match_ritis_events(events_within_counties, sites_within_counties, radius=500, predicate='within')

CRS for events: EPSG:4326
CRS for sites: EPSG:4326
Size of events_gdf with CCS_Pair:  (117936, 42)


#### Get the event counts for each corresponding site and add it to the sites DF

In [25]:
event_count = matched_events.groupby('CCS_Pair').size().reset_index(name='event_count')

In [27]:
sites_within_counties_matched = pd.merge(sites_within_counties, event_count, left_on='TC_NUM', right_on='CCS_Pair', how='left').dropna(subset='event_count')
sites_within_counties_matched['event_count'] = sites_within_counties_matched['event_count'].astype('int')

### Creating interactive map 

In [331]:
###test code for troubleshooting issues

m = folium.Map(location=[0, 0], zoom_start=6)

m.add_child(folium.FeatureGroup(name='test'))
folium.TileLayer('CartoDB positron').add_to(m)

mcg = folium.plugins.MarkerCluster(control=False, name='test2')
m.add_child(mcg)

g1 = folium.plugins.FeatureGroupSubGroup(mcg, "Less than 4 Events")
g2 = folium.plugins.FeatureGroupSubGroup(mcg, "Fire")
g3 = folium.plugins.FeatureGroupSubGroup(mcg, "Accident")

test_dict = {'g1':g1, 'g2':g2, 'g3':g3}

folium.Marker([-1, -1]).add_to(g1)
folium.Marker([1, 1]).add_to(g1)

folium.Marker([-1, 1]).add_to(g2)
folium.Marker([1, -1]).add_to(g2)

folium.Marker([1, 2]).add_to(g3)
folium.Marker([-1, -2]).add_to(g3)

for hehe, test_group in test_dict.items():
    m.add_child(test_group)

folium.LayerControl().add_to(m)

GroupedLayerControl(groups={'Events':[test_dict[test] for test in test_dict]}, exclusive_groups=False, collapsed=False).add_to(m)

m

#### import libraries

In [341]:
import folium
from folium.plugins import MarkerCluster, GroupedLayerControl, FeatureGroupSubGroup

#### define style functions for coloring etc.

In [342]:
def style_function1(feature):
    return {'fillColor':'light gray','color':'#000000', 'weight':'1'}

def style_function2(feature):
    return {'fillColor':'yellow','color':'gray', 'weight':'1'}

#### read in Georgia Shape file and South Georgia Shape Files to determine boundaries and geographic locations

In [343]:
ga_shp = gpd.read_file('D:/OneDrive - University of Georgia/RP_22-35_Area-Specific_LDF/data/GA Map Data and Poly Files/US/tl_2020_us_state.shp')
ga_geom = ga_shp[ga_shp['NAME'] == 'Georgia']['geometry'].iloc[0]

south_georgia = gpd.read_file(shp_path)
south_georgia = south_georgia[['geometry']]

south_gajson = south_georgia.to_crs(epsg='4326').to_json()

#### create base map

In [344]:
map = folium.Map(location=[sites_within_counties['LAT'].mean(), sites_within_counties['LONG'].mean()], zoom_start=9)

#### Define feature groups and clusters

In [345]:
GA = folium.FeatureGroup(name='Georgia')
southGA = folium.FeatureGroup(name='South Georgia')

site_cluster = MarkerCluster(control=False)
event_cluster = MarkerCluster(control=False)

#### define unique event types/agencies/durations in order to have customized layer control for interactive map

In [346]:
event_specific_types = {}

specific_types = matched_events['Agency-specific Type'].unique()

event_count_groups = {
    'less_than_50': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Less than 50 events'),
    'between_50_and_100': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 50-100 events'),
    'between_101_and_200': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 101-200 events'),
    'between_201_and_300': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 201-300 events'),
    'between_301_and_400': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 301-400 events'),
    'between_401_and_500': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 401-500 events'),
    'between_501_and_600': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 501-600 events'),
    'between_601_and_700': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 601-700 events'),
    'between_701_and_800': folium.plugins.FeatureGroupSubGroup(site_cluster, 'Between 701-800 events'),
    'more_than_801': folium.plugins.FeatureGroupSubGroup(site_cluster, 'More than 801 Events')
}

for specific_type in specific_types:
    specific_type_key = f'{specific_type}'
    event_specific_types[specific_type_key] = folium.plugins.FeatureGroupSubGroup(event_cluster, f'{specific_type}')

In [347]:
for index, site in sites_within_counties_matched.iterrows():

    site_id = site['TC_NUM']
    county = site['Label']
    no_lanes = site['TOTAL_LANE']
    area = site['Urban_Rural']
    f_class = site['F_SYSTEM']
    events = site['event_count']

    popup_content_site = f'<b>Site ID: {site_id}</b><br>Total Events Matched: {events}<br>County: {county}<br># of Lanes (total): {no_lanes}<br>Area: {area}<br>Facility Type (classification): {f_class}'

    site_marker = folium.Marker([site['LAT'], site['LONG']], tooltip=f'Site {site_id}', icon=folium.Icon(icon='car', prefix='fa',color='black'), popup=popup_content_site)

    site_popup = folium.Popup(popup_content_site, max_width=400)
    site_marker.add_child(site_popup)

    # Determine the correct event count group
    if events < 50:
        event_count_key = 'less_than_50'
    elif 50 <= events <= 100:
        event_count_key = 'between_50_and_100'
    elif 101 <= events <= 200:
        event_count_key = 'between_101_and_200'
    elif 201 <= events <= 300:
        event_count_key = 'between_201_and_300'
    elif 301 <= events <= 400:
        event_count_key = 'between_301_and_400'
    elif 401 <= events <= 500:
        event_count_key = 'between_401_and_500'
    elif 501 <= events <= 600:
        event_count_key = 'between_501_and_600'
    elif 601 <= events <= 700:
        event_count_key = 'between_601_and_700'
    elif 701 <= events <= 800:
        event_count_key = 'between_701_and_800'
    else:
        event_count_key = 'more_than_801'

    site_marker.add_to(event_count_groups[event_count_key])

for _, event in matched_events.iterrows():

    identifier = event['Event ID']
    type = event['Standardized Type']
    specific_type = event['Agency-specific Type']
    specific_subtype = event['Agency-specific Sub Type']
    duration = event['Duration_minutes']
    agency = event['Agency']
    ccs = event['CCS_Pair']

    popup_content_event = f'<b>Crash ID: {identifier}</b><br>Agency: {agency}<br>Type: {type}<br>Subtype: {specific_type}<br>Duration: {duration}<br>CCS station pair: {ccs}'

    event_marker = folium.CircleMarker([event['Latitude'], event['Longitude']], radius=4, popup=popup_content_event, tooltip=f'Event {identifier}', color='red')
    
    # Determine the correct duration group
    if duration < 30:
        duration_key = 'less_than_30'
    elif 30 <= duration <= 60:
        duration_key = 'between_30_and_60'
    else:
        duration_key = 'more_than_60'

    event_popup = folium.Popup(popup_content_event, max_width=400)
    event_marker.add_child(event_popup)

    event_marker.add_to(event_specific_types[specific_type])

#### add layer control

In [348]:
folium.TileLayer('CartoDB positron').add_to(map)

folium.GeoJson(ga_geom, style_function=style_function1).add_to(GA)
folium.GeoJson(south_georgia, style_function=style_function2).add_to(southGA)

GA.add_to(map)
southGA.add_to(map)
map.add_child(site_cluster)
map.add_child(event_cluster)

for event_type, subgroup in event_specific_types.items():
    map.add_child(subgroup)

for site, count_group in event_count_groups.items():
    map.add_child(count_group)

folium.LayerControl(collapsed=True).add_to(map)

GroupedLayerControl(
    groups={'Specific Event Types': [event_specific_types[specific_event] for specific_event in event_specific_types],
            'Portable CCS Sites': [event_count_groups[event_count_group] for event_count_group in event_count_groups]
            },
    collapsed=False,
    exclusive_groups=False
    ).add_to(map)

#### display map

In [5]:
#map

#### save map

In [350]:
map.save(r'D:\OneDrive - University of Georgia\Research\Fall 2023\RP 23-25 - Probe Project\Probe Sites Interactive Map.html')